In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
print(tf.__version__)


2025-09-01 08:27:38.248161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756715258.446698      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756715258.502788      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.0


In [2]:
!pip install mne numpy scipy scikit-learn matplotlib


In [4]:
import numpy as np
import scipy.io
from scipy.signal import hilbert
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [ ]:
print(31066023 / 22)  # Should be an integer (number of samples)


In [5]:
import numpy as np
from scipy.signal import hilbert
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from scipy.stats import mode
import matplotlib.pyplot as plt


In [9]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def load_csv_folder(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = [pd.read_csv(f) for f in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

train_folder = '/kaggle/input/Alcoholics/SMNI_CMI_TRAIN'
test_folder = '/kaggle/input/Alcoholics/SMNI_CMI_TEST'

train_df = load_csv_folder(train_folder)
test_df = load_csv_folder(test_folder)

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

# Assuming EEG channels columns are known, e.g.
eeg_cols = ['C3', 'C4', 'CZ']  # replace with your actual EEG channel column names

X_train = train_df[eeg_cols].values
y_train_raw = train_df['subject identifier'].values  # exact column name might vary

X_test = test_df[eeg_cols].values
y_test_raw = test_df['subject identifier'].values


Train data shape: (7667712, 10)
Test data shape: (7864320, 10)


KeyError: "None of [Index(['C3', 'C4', 'CZ'], dtype='object')] are in the [columns]"

In [10]:
print(train_df.columns)
print(test_df.columns)


Index(['Unnamed: 0', 'trial number', 'sensor position', 'sample num',
       'sensor value', 'subject identifier', 'matching condition', 'channel',
       'name', 'time'],
      dtype='object')
Index(['Unnamed: 0', 'trial number', 'sensor position', 'sample num',
       'sensor value', 'subject identifier', 'matching condition', 'channel',
       'name', 'time'],
      dtype='object')


In [11]:
# Make sure 'sample num' and 'channel' uniquely identify each waveform point for each trial
eeg_df = train_df.pivot_table(
    index=['subject identifier', 'trial number', 'sample num'],   # rows: each point in each trial
    columns='sensor position',                                    # columns: each sensor/channel
    values='sensor value'                                         # values: signal at that sample
)

eeg_df = eeg_df.reset_index()
print(eeg_df.head())
print("EEG pivot shape:", eeg_df.shape)


sensor position subject identifier  trial number  sample num       AF1  \
0                                a             0           0 -0.832714   
1                                a             0           1 -0.832714   
2                                a             0           2 -0.344286   
3                                a             0           3  0.143714   
4                                a             0           4  1.329857   

sensor position       AF2       AF7       AF8       AFZ        C1        C2  \
0               -0.074143 -3.396143 -0.453429 -0.357429  0.011571  1.062143   
1                0.135286 -1.791714  0.244286 -0.427143 -0.197571  0.225000   
2                0.274571  1.277429  1.430000 -0.008857  0.290714  0.225143   
3                0.414286  4.416286  2.755286  0.410000  0.499857 -0.053857   
4                1.181429  6.020571  3.522571  1.316714  0.778857  0.364714   

sensor position  ...       PO8       POZ        PZ        T7        T8  \
0     

In [12]:
eeg_cols = eeg_df.columns[eeg_df.columns.get_loc('sample num')+1:]  # all sensor columns after time indices
arr_eeg = eeg_df[eeg_cols].values  # shape: (num_points, num_channels)
raw_labels = eeg_df['subject identifier'].values


In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
subject_labels = le.fit_transform(raw_labels)  # integer label for each EEG point/window
print("Subject label classes:", le.classes_)


Subject label classes: ['a' 'c']


In [16]:
# raw_labels is your array of subject identifier strings
import numpy as np

label_map = {'a': 1, 'c': 0}
subject_labels = np.array([label_map[lbl] for lbl in raw_labels])
print(np.unique(subject_labels, return_counts=True))  # Should print (array([0, 1]), counts)


(array([0, 1]), array([15616, 11776]))


In [22]:
def sequence_labels(windowed_labels, time_steps):
    seq_labels = []
    for i in range(len(windowed_labels) - time_steps + 1):
        vals, counts = np.unique(windowed_labels[i:i+time_steps], return_counts=True)
        seq_label = vals[np.argmax(counts)]
        seq_labels.append(seq_label)
    return np.array(seq_labels)


In [33]:
# Make sure segments is a NumPy array
segments = np.array(segments)

# Now you can safely get shape
n_channels = segments.shape[-1]  # segments shape: (num_segments, window_size, n_channels)
input_shape = (time_steps, n_channels, n_channels, 1)

print("n_channels:", n_channels)
print("input_shape:", input_shape)


n_channels: 64
input_shape: (3, 64, 64, 1)


In [34]:
print(eeg_data.shape)  # Should be (total_samples_in_trial, n_channels)



(256, 64)


In [35]:
segment = eeg_data[start:start+win_size, :]
print("Segment shape:", segment.shape)


Segment shape: (128, 64)


In [37]:
segments = []
for start in range(0, len(eeg_data) - win_size + 1, step_size):
    segment = eeg_data[start:start+win_size, :]
    segments.append(segment)  # THIS LINE IS CRUCIAL
    print("Segment shape:", segment.shape)


Segment shape: (128, 64)
Segment shape: (128, 64)
Segment shape: (128, 64)


In [38]:
segments = np.array(segments)
print("Segments array shape:", segments.shape)
n_channels = segments.shape[-1]
print("n_channels:", n_channels)


Segments array shape: (3, 128, 64)
n_channels: 64


In [39]:
print("Number of segments extracted:", len(segments))


Number of segments extracted: 3


In [40]:
X_plv = np.stack([plv_matrix(seg) for seg in segments])  # Shape: (3, 64, 64)
print("PLV data shape:", X_plv.shape)


PLV data shape: (3, 64, 64)


In [41]:
def create_sequences(data, time_steps):
    return np.array([data[i:i+time_steps] for i in range(len(data) - time_steps + 1)])

time_steps = 3  # or 2, depending how many per sequence you want
X_seq = create_sequences(X_plv, time_steps)  # shape: (1, 3, 64, 64)
X_seq = X_seq[..., np.newaxis]  # (1, 3, 64, 64, 1)
print("X_seq shape:", X_seq.shape)


X_seq shape: (1, 3, 64, 64, 1)


In [42]:
input_shape = (time_steps, n_channels, n_channels, 1)
print("Model input_shape:", input_shape)


Model input_shape: (3, 64, 64, 1)


In [43]:
from tensorflow.keras import layers, models, Input

input_shape = (time_steps, n_channels, n_channels, 1)  # (3, 64, 64, 1)
inputs = Input(shape=input_shape)

# 2D CNN applied at each time step via TimeDistributed
x = layers.TimeDistributed(
        layers.Conv2D(16, (3, 3), padding='same', activation='relu'))(inputs)
x = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(x)
x = layers.TimeDistributed(layers.Flatten())(x)

# Temporal modeling via BiLSTM
x = layers.Bidirectional(
        layers.LSTM(32, return_sequences=False)
    )(x)

# Final classification (binary)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


I0000 00:00:1756715916.172212      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756715916.172828      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3, 64, 64, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 3, 64, 64, 16)  │           160 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 3, 32, 32, 16)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 3, 16384)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │     4,202,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,202,977 (16.03 MB)

 Trainable params: 4,202,977 (16.03 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# 1. Initialize storage lists
X_seq_list = []
y_seq_list = []

# 2. Loop: for each trial, segment, compute PLV, build sequence and label(s)
trial_nums = eeg_df['trial number'].unique()
for trial_num in trial_nums:
    trial_mask = eeg_df['trial number'] == trial_num
    eeg_trial = eeg_df.loc[trial_mask, eeg_cols].values
    subj_label = eeg_df.loc[trial_mask, 'subject identifier'].values[0]
    mapped_label = label_map[subj_label]

    # a. Extract segments
    segments = []
    for start in range(0, len(eeg_trial) - win_size + 1, step_size):
        segment = eeg_trial[start:start+win_size, :]
        segments.append(segment)
    segments = np.array(segments)

    # b. Skip trials without enough windows for a sequence
    if len(segments) < time_steps:
        continue

    # c. Compute PLV for each window segment
    X_plv_trial = np.stack([plv_matrix(seg) for seg in segments])  # shape: (num_segments, 64, 64)

    # d. Create time sequences
    X_seq_trial = create_sequences(X_plv_trial, time_steps)  # shape: (num_sequences, time_steps, 64, 64)
    X_seq_trial = X_seq_trial[..., np.newaxis]               # shape: (num_sequences, time_steps, 64, 64, 1)

    # e. Make corresponding label for each sequence
    y_seq_trial = np.full((X_seq_trial.shape[0],), mapped_label)

    # f. Append to master lists
    X_seq_list.append(X_seq_trial)
    y_seq_list.append(y_seq_trial)

# 3. Concatenate all trials
X_seq_all = np.concatenate(X_seq_list, axis=0)
y_seq_all = np.concatenate(y_seq_list, axis=0)

print("Final data shapes:", X_seq_all.shape, y_seq_all.shape)

# 4. Train model
history = model.fit(X_seq_all, y_seq_all, batch_size=8, epochs=20, validation_split=0.2)


Final data shapes: (242, 3, 64, 64, 1) (242,)
Epoch 1/20


I0000 00:00:1756715944.090760     122 cuda_dnn.cc:529] Loaded cuDNN version 90300


25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.8701 - loss: 0.1091 - val_accuracy: 0.6735 - val_loss: 2.1155
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.6735 - val_loss: 2.2331
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 9.9926e-04 - val_accuracy: 0.6735 - val_loss: 2.3274
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 7.5507e-04 - val_accuracy: 0.6735 - val_loss: 2.4094
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 5.9187e-04 - val_accuracy: 0.6735 - val_loss: 2.4812
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 4.7783e-04 - val_accuracy: 0.6735 - val_loss: 2.5449
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 3.9502e-04 - val_accuracy: 0.6735 - val_loss: 2.6020
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 3.3300e-04 - val_accurac

In [59]:
from sklearn.model_selection import train_test_split

# Split trials between train and test (e.g., 80% train, 20% test by trial)
trial_nums = eeg_df['trial number'].unique()
train_trials, test_trials = train_test_split(trial_nums, test_size=0.2, random_state=42)

eeg_df_train = eeg_df[eeg_df['trial number'].isin(train_trials)].copy()
eeg_df_test = eeg_df[eeg_df['trial number'].isin(test_trials)].copy()

test_trial_nums = eeg_df_test['trial number'].unique()


In [60]:
import pandas as pd

# Example: loading from CSV
# eeg_df_test = pd.read_csv('path_to_your_test_file.csv')
# OR, if you already have another DataFrame just use the correct variable name!

test_trial_nums = eeg_df_test['trial number'].unique()


In [51]:
# Initialize lists for test sequences and labels
segments_test = []
labels_test = []

# Unique trial numbers in test set
test_trial_nums = eeg_df_test['trial number'].unique()

for trial_num in test_trial_nums:
    trial_mask = eeg_df_test['trial number'] == trial_num
    eeg_trial = eeg_df_test.loc[trial_mask, eeg_cols].values
    subj_label = eeg_df_test.loc[trial_mask, 'subject identifier'].values[0]
    mapped_label = label_map[subj_label]
    
    # Segment the trial with same window and step sizes as training
    trial_segments = []
    for start in range(0, len(eeg_trial) - win_size + 1, step_size):
        segment = eeg_trial[start:start+win_size, :]
        trial_segments.append(segment)
    
    if len(trial_segments) < time_steps:
        continue
    
    # Compute PLV matrix for each segment
    X_plv_trial = np.stack([plv_matrix(seg) for seg in trial_segments])
    
    # Create time sequences (length = time_steps)
    X_seq_trial = create_sequences(X_plv_trial, time_steps)[..., np.newaxis]  # add CNN channel dim
    
    # Label sequence with same mapped label
    y_seq_trial = np.full((X_seq_trial.shape[0],), mapped_label)
    
    segments_test.append(X_seq_trial)
    labels_test.append(y_seq_trial)

# Concatenate all test sequences and labels
X_test = np.concatenate(segments_test, axis=0)
y_test = np.concatenate(labels_test, axis=0)

print("Test data shape (X_test):", X_test.shape)     # Should print (?, 3, 64, 64, 1)
print("Test labels shape (y_test):", y_test.shape)


Test data shape (X_test): (41, 3, 64, 64, 1)
Test labels shape (y_test): (41,)


In [52]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9024 - loss: 0.9125    
Test loss: 1.3687, Test accuracy: 0.8537


In [53]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict (for binary, threshold at 0.5)
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 402ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.85      1.00      0.92        35

    accuracy                           0.85        41
   macro avg       0.43      0.50      0.46        41
weighted avg       0.73      0.85      0.79        41

Confusion Matrix:
[[ 0  6]
 [ 0 35]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Validation Acc')
plt.legend(); plt.title('Accuracy'); plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(); plt.title('Loss'); plt.show()


In [54]:
model.save('plv_2dcnn_bilstm_model.h5')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predict labels (assuming binary classification threshold 0.5)
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

model_name = "PLV 2DCNN BiLSTM"

y_pred = (model.predict(X_test) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(4, 3))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues, ax=plt.gca())

plt.title(f"Confusion Matrix - {model_name}")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict labels (binary threshold 0.5)
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Get classification report as dict
report_dict = classification_report(y_test, y_pred, output_dict=True)

# Function to format floats in dict to 4 decimals
def format_report(d):
    for key, value in d.items():
        if isinstance(value, dict):
            format_report(value)
        else:
            if isinstance(value, float):
                d[key] = f"{value:.4f}"
    return d

formatted_report = format_report(report_dict)

# Print formatted report nicely
for label, metrics in formatted_report.items():
    if isinstance(metrics, dict):
        print(f"\n{label}:")
        for metric, val in metrics.items():
            print(f"  {metric}: {val}")
    else:
        print(f"{label}: {metrics}")

# Print confusion matrix normally
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Validation Acc')
plt.legend()
plt.title('Accuracy')
plt.tight_layout()
plt.show()

plt.figure(figsize=(4, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.tight_layout()
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

output_dir = '/kaggle/working/plv_heatmaps'
os.makedirs(output_dir, exist_ok=True)

def plot_and_save_plv_matrix(plv_matrix, title, filename):
    plt.figure(figsize=(5, 5))
    sns.heatmap(plv_matrix, cmap='coolwarm', vmin=0, vmax=1, square=True)
    plt.title(title)
    plt.xlabel('Channels')
    plt.ylabel('Channels')
    plt.colorbar(label='PLV')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename))
    plt.show()
    plt.close()
    print(f'Saved plot: {filename}')

# Example usage: plot single sample
plot_and_save_plv_matrix(
    X_plv[0], 
    'Sample PLV Matrix - Segment 0',
    'sample_plv_segment0.png'
)


In [57]:
# Example: assign label per segment (customize as per your data)
# Assuming segment 0 and 2 are alcoholic (label=1), segment 1 is control (label=0)

segment_labels = np.array([1, 0, 1])


In [58]:
import numpy as np

avg_plv_alcoholic = np.mean(X_plv[segment_labels == 1], axis=0)
avg_plv_control = np.mean(X_plv[segment_labels == 0], axis=0)


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

output_dir = '/kaggle/working/plv_heatmaps'
os.makedirs(output_dir, exist_ok=True)

def plot_and_save_plv_matrix(plv_matrix, title, filename):
    plt.figure(figsize=(5, 5))
    sns.heatmap(plv_matrix, cmap='coolwarm', vmin=0, vmax=1, square=True)
    plt.title(title)
    plt.xlabel('Channels')
    plt.ylabel('Channels')
    plt.colorbar(label='PLV')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, filename))
    plt.show()
    plt.close()
    print(f'Saved plot: {filename}')

plot_and_save_plv_matrix(
    avg_plv_alcoholic, 'Average PLV Matrix - Alcoholic Group', 'avg_plv_alcoholic.png'
)

plot_and_save_plv_matrix(
    avg_plv_control, 'Average PLV Matrix - Control Group', 'avg_plv_control.png'
)


In [ ]:
plot_and_save_plv_matrix(
    X_plv[0], 'Sample PLV Matrix - Segment 0', 'sample_plv_segment0.png'
)


In [ ]:
# Show alcoholic average PLV
plot_and_save_plv_matrix(
    avg_plv_alcoholic, 
    'Average PLV Matrix - Alcoholic Group', 
    'avg_plv_alcoholic.png'
)

# Show control average PLV
plot_and_save_plv_matrix(
    avg_plv_control, 
    'Average PLV Matrix - Control Group', 
    'avg_plv_control.png'
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

output_dir = '/kaggle/working/plv_heatmaps'
os.makedirs(output_dir, exist_ok=True)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.heatmap(avg_plv_alcoholic, cmap='coolwarm', vmin=0, vmax=1, square=True)
plt.title('Average PLV Matrix - Alcoholic Group')
plt.xlabel('Channels')
plt.ylabel('Channels')

plt.subplot(1, 2, 2)
sns.heatmap(avg_plv_control, cmap='coolwarm', vmin=0, vmax=1, square=True)
plt.title('Average PLV Matrix - Control Group')
plt.xlabel('Channels')
plt.ylabel('Channels')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'avg_plv_alcoholic_vs_control.png'))
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Create output directory if it doesn't exist
output_dir = '/kaggle/working/plv_heatmaps'
os.makedirs(output_dir, exist_ok=True)

def plot_and_save_plv_sequence(plv_seq, indices, filename_prefix):
    plt.figure(figsize=(5 * len(indices), 5))
    for i, idx in enumerate(indices):
        plt.subplot(1, len(indices), i + 1)
        sns.heatmap(plv_seq[idx], cmap='coolwarm', vmin=0, vmax=1, square=True)
        plt.title(f'Segment {idx}')
        plt.axis('off')
    plt.tight_layout()
    plt.suptitle('Temporal Evolution of PLV Connectivity')
    plt.subplots_adjust(top=0.85)
    
    filename = f'{filename_prefix}_plv_sequence.png'
    plt.savefig(os.path.join(output_dir, filename))
    plt.show()
    plt.close()
    print(f'Saved plot: {filename}')

# Example usage: plot first 3 segments in sequence
plot_and_save_plv_sequence(X_plv, [0, 1, 2], 'temporal_evolution')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# All off-diagonal PLV values from average matrices
alco_vals = avg_plv_alcoholic[np.triu_indices_from(avg_plv_alcoholic, k=1)]
ctrl_vals = avg_plv_control[np.triu_indices_from(avg_plv_control, k=1)]

plt.figure(figsize=(8, 5))
plt.hist(alco_vals, bins=30, alpha=0.6, label='Alcoholic', color='r', density=True)
plt.hist(ctrl_vals, bins=30, alpha=0.6, label='Control', color='b', density=True)
plt.xlabel('PLV Value')
plt.ylabel('Density')
plt.title('Distribution of Off-Diagonal PLV Values')
plt.legend()
plt.tight_layout()
plt.savefig('/kaggle/working/plv_heatmaps/plv_distribution_hist.png')
plt.show()
